# Scrapes 5000 most common French words from this site
https://www.memrise.com/course/131111/5000-most-common-french-words

They are divided into 50 pages of 100 words each. This script quickly goes and scrapes them all

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import os

In [2]:
all_words = []
for page in tqdm(range(1, 51)):
    
    # Wait 1s
    time.sleep(1)
    
    # Get url
    url = f'https://www.memrise.com/course/131111/5000-most-common-french-words/{page}/'
    response = requests.get(url)
    
    # Parse url
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find the table and extract the rows
    table = soup.find_all("div", attrs={"class": "things clearfix"})
    if len(table) == 1:
        table = table[0]
    else:
        raise RuntimeError("multiple tables found")
        
    for row in table.find_all("div", attrs={"class": "thing text-text"}):
        words = []
        i = 0
        for entry in row.find_all("div", attrs={"class": "text"}, recursive=False):
            words.append(entry.text)
        assert len(words) == 2, "More than 2 words found: aborting!"
        all_words.append(tuple(words))

In [3]:
len(all_words)  # should be 5000

5000

In [4]:
# Make sure output looks ok
all_words[0:5]

[('un', 'a, an, one'),
 ('à', 'to, at, in [not "chez"]'),
 ('en', 'in, by'),
 ('le', 'the; him, her, it, them'),
 ('et', 'and')]

In [5]:
# Write into a tsv with an extra column for "add reverse card" since my cards have that
# If you don't want the reverse card, just remove the y
with open(os.environ["HOME"] + "/Downloads/fren.txt", "w") as f:
    for fr, en in all_words:
        f.write(f"{fr}\t{en}\ty\n")